In [1]:
!pip install --quiet -r rag_system/requirements.txt

You should consider upgrading via the '/Users/nicolas.dominutti/Desktop/ml/medical-qa-system/.venv/bin/python3.10 -m pip install --upgrade pip' command.


In [2]:
from dotenv import load_dotenv
load_dotenv("rag_system/.env")
MODEL_PATH = 'rag_system/model'

In [3]:
from general_utils import load_config
CONFIG = load_config()

In [4]:
from general_utils import S3Manager
s3_client = S3Manager.get_client('s3')

In [ ]:
S3Manager.download_folder(s3_client, CONFIG['S3_BUCKET'], 'finetuned_model/', MODEL_PATH)

---

In [5]:
from general_utils.flagembedding import FlagEmbeddingManager
flagg_embedding = FlagEmbeddingManager()
model = flagg_embedding.get_model(
    local_model_path=MODEL_PATH,
    query_instruction=CONFIG['QUERY_INSTRUCTION_AT_RETRIEVAL']
)

/Users/nicolas.dominutti/Desktop/ml/medical-qa-system/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from rag_system.utils import BedrockManager
bedrock = BedrockManager()

---

In [9]:
def chat(query):
    context = flagg_embedding.search(
        endpoint_url=CONFIG['OPENSEARCH_INDEX_URL'],
        flag_embedding_model=model, 
        query=query,
        top_k=CONFIG['ITEMS_TO_RETRIEVE']
    )
    return bedrock.ask(
        query,
        context,
        CONFIG['MODEL'],
        CONFIG['SYSTEM_PROMPT'],
        CONFIG['TEMPERATURE'],
        CONFIG['TOP_P'],
        CONFIG['MAX_OUTPUT_TOKENS']
    )

In [ ]:
res = chat('What is high blood pressure?')


In [ ]:
print(response['generation'])

 High blood pressure, also known as hypertension, is a condition where the force of blood pushing against the walls of the blood vessels is higher than normal. It occurs when your blood pressure rises and stays high over time, making the heart work too hard and potentially harming arteries and organs such as the heart, kidneys, brain, and eyes.
